# Data 301 Final Project Analysis

The two questions leading to below analysis is more exploratory than descriptive or predictive:

    How positive are the users of each Premier League Subreddit throughout a season?
    
    How does each soccer subreddit uniquely behave?

In order to answer this question, we'll use the Reddit Comments dataset, hosted on Google BigQuery, and import the data into a local database. We'll then analyze each comment using the NLTK Sentiment Intensity Analyzer and assign each comment a positivity and negativity value. Using the sentiment scores, we can then aggregate over each month and week to see if the subreddit's positivity and negativity fluctuate with the team's results or news. Finally, we'll attempt to build a TD-IDF K-Nearest-Neighbors model in order to predict the score of each comment. The reddit comment score is some function of upvotes, downvotes, and the age of the comment.

## Retrieval from Google BigQuery

After looking through [this reddit post](https://www.reddit.com/r/datasets/comments/3bxlg7/i_have_every_publicly_available_reddit_comment/), I found the Reddit comments dataset on [Google BigQuery](https://cloud.google.com/bigquery/). As a student, I was able to create a free, one-year account that has $300 worth of compute credits.

Then, I decided to find the subreddits associated with the clubs in the top four tiers of English soccer. I was able to find a majority of these by looking at [soccer-related subreddits](https://www.reddit.com/r/soccer/wiki/relatedsubreddits) and cross-referencing it with a list of teams in each league. I compiled these into a .csv file and uploaded them to BigQuery as a standalone table.

Since I had a list of subreddits I cared about, I could query the Reddit Comments table and filter out any comments from subreddits that weren't in my list. As a result of some data exploration I did in the BigQuery console, I also filtered out comments that were deleted, and ended up only retrieving comments from Premier League subreddits. This last choice was after realizing that the non-Premier League subreddits are very small (100-500 total users) and relatively inactive (only a few daily active users), making textual analysis almost worthless for those subreddits.

Also, I decided to retrieve only specific attributes of the Comment dataset: body, subreddit, author, score, ups, downs, and created_utc. All of the other fields are either rarely populated or irrelevant to the analysis I'm doing.

![Comment Query](CommentQuery.png)

The result of this query is too large for Google BigQuery to return on the console, so I had to export the results to a Google Cloud Storage bucket. By deciding the prefix of each file, the later reading of the data was fairly simple. Once in the storage bucket, I downloaded each .csv file locally and began the import process.

## Importing Comment Data

Using these files, I created a local MySQL database and imported the comment data. The reason that I had to import to a local MySQL table is because of the size of this dataset and the poor hardware specs of my laptop.

I performed a series of filtering and transformations before importing the data, however.
* The comment data contains unicode characters, such as emojis, so I had to filter those characters out.
* I converted the date field from milliseconds to a datetime object. Thinking back, this probably slowed down the import process significantly, and I could have just converted the date in the retrieval MySQL queries rather than store millions of datetime objects.
* I unescaped HTML characters (`&amp;` should be `&`) and then removed URLs from each comment. These should never affect the sentiment analyzer, as well as saving a little bit of space in the database.
* I filtered out any comments made before June 1, 2017, so that I could look at a single Premier League season, as well as any comments that were removed.

Once these transformations and filters were written, I wrote a helper method that took in a .csv filename and inserted all valid rows contained in the file into the local MySQL server.

After all of the files were processed, I added an index to the subreddit field so that I could easily retrieve comments for each subreddit.

In [1]:
file_prefix = "/mnt/c/Users/Brent Williams/Downloads/"
actuallyRun = False

for x in range(0, 7, 1):
    if actuallyRun:
        filename = file_prefix + "comments" + str(x).zfill(12)
        read_and_insert(filename)
        finished_files.append(x)

All in all, I ended up with 3.67 million reddit comments, all from the past year. This data can be used to determine which subreddits were the most positive, or the most negative, at different parts of the season.

## Compute Sentiment Analysis Scores

By using the NLTK Sentiment Intensity Analyzer, I was able to computer the positivity and negativity score for each given comment. This class weights punctuation and upper/lowercase differently, which is why I didn't simplify the original comments completely.

Similar to the above process, I created a MySQL table to contain all the results of the sentiment analysis so that my computer wouldn't crash, and I looped through the contents of the Comments table. By performing a left join on the comments table, I was able to determine which comments had already been analyzed as I iterated through the table, 140,000 comments at a time.

Once finished, I added a foreign key constraint from the Comments Metadata table to the Comments table to ensure that I didn't have any rogue comments. I also manually checked that the table had the same number of rows as the comments table.

In [2]:
import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    import nltk
    from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

def sentimentAnalysis(comment):
    return sid.polarity_scores(comment)

In [3]:
sentimentAnalysis("This comment sucks, it's the WORST!")

{'compound': -0.8236, 'neg': 0.656, 'neu': 0.344, 'pos': 0.0}

In [4]:
sentimentAnalysis("This comment is awesome, and it's the BEST!")

{'compound': 0.884, 'neg': 0.0, 'neu': 0.392, 'pos': 0.608}

## Sentiment Analysis by Subreddit

After performing the sentiment analysis for each comment, I had to actually do something with it. I looked at the sentiment scores for each subreddit over each month.

Because of the large amount of data in the dataset, this aggregation logic is much faster to perform in MySQL, rather than with Pandas.

MySQL Query:
```
SELECT  c.subreddit,
        YEAR(c.created_utc) as year,
        MONTH(c.created_utc) as month,
        AVG(cm.positivity) as positivity,
        AVG(cm.negativity) as negativity,
        AVG(cm.net_score) as average_score
FROM    comments c
    LEFT JOIN
        comments_meta cm USING (id)
GROUP BY c.subreddit, YEAR(c.created_utc), MONTH(c.created_utc)
```

Equivalent Pandas logic:
```
groupBy = [comments['subreddit'], comments['created_utc'].map(lambda x: (x.year, x.month))]
positivityByDay = comments.groupby(by=groupBy)[['net_score']].mean()
```

Example Graph:

![Example By Month Graph](ExampleByMonthGraph.png)

I also wanted to look at sentiment scores for each week, since a team's monthly sentiment score will be relatively constant. A bad loss followed by a good win should register as a neutral month. However, by grouping per week, we should be able to tell when a team suffers a loss or defeats a rival.

MySQL Query:
```
SELECT  c.subreddit,
        YEAR(c.created_utc) as year,
        MONTH(c.created_utc) as month,
        DAY(c.created_utc) as day,
        AVG(cm.positivity) as positivity,
        AVG(cm.negativity) as negativity,
        AVG(cm.net_score) as average_score
FROM    comments c
    LEFT JOIN
        comments_meta cm USING (id)
GROUP BY c.subreddit, YEAR(c.created_utc), MONTH(c.created_utc), WEEK(c.created_utc)
ORDER BY c.created_utc DESC
```

Equivalent Pandas logic:
```
groupBy = [comments['subreddit'], comments['created_utc'].map(lambda x: (x.year, x.month, x.week))]
positivityByDay = comments.groupby(by=groupBy)[['net_score']].mean()
```

Example Graph:

![Example By Week Graph](ExampleByWeekGraph.png)

Since this graph is over a smaller time period, it's more sensitive to day-to-day news and results. To prove this, I'll take a known upsetting result and see if there's a change in positivity or negativity in the graph.

For example, Liverpool lost to Tottenham 4-1 on October 22nd, 2017. Let's draw a dashed line at this date.

![Upsetting Result](UpsettingResult.png)

As we can see in the graph, the week of the loss against Tottenham saw a big spike in negativity, increasing by approximately 0.03 from the previous week, as well as a small drop in positive comments. The net score line, in yellow, drops from almost 0.08 the previous week to 0.03 after the loss to Tottenham.

For each subreddit, I combined both of the above graphs and tried to figure out what caused increases in positivity or negativity. I won't show all of those here, but let's look at a big upset: Huddersfield Town, a newly promoted team, beat Manchester United 2-1 on October 22nd|.

![Huddersfield](Huddersfield.png)

The spike in positivity in the subreddit is quite obvious! Their negativity score went didn't change from the previous week, while their positivity score went up from around 0.15 to 0.4!

Now, let's look at Manchester United's subreddit at the same time.

![United](united.png)

This negativity spike is just as obvious as Huddersfield Town's positive spike. The Manchester United subreddit went down around 0.01 in positivity, while also increasing around 0.04 points in negativity. This change is easiest seen in the net score graph, going from around 0.07 down to 0.03 or 0.02.

## Sentiment During a Game

I was curious to see if the dataset provided, combined with the sentiment analysis by NLTK, contains enough information for me to see a subreddit's mentality change during a game. To visualize this, let's look at a game that provided plenty of emotional switches: Liverpool v. Arsenal on December 22, 2017. As mentioned earlier, Liverpool took an earlier lead, then conceded three goals in the space of five minutes. The increase in negativity is obvious on a week-by-week scale, but I wondered if we could see it throughout the match.

In order to test this theory, I retrieved all comments from the dataset that occurred on December 22, then filtered the comments even further using a boolean mask to only retrieve comments from 15 minutes before kickoff to 15 minutes after the match ended. I then split the comments into Liverpool comments and Arsenal comments, throwing away any others, and aggregated the net positivity score on a minute-by-minute basis.

For ease of visualization, I drew vertical lines indicating a goal. The light pink lines are Liverpool goals, while the light yellow lines are Arsenal goals.

![Liverpool vs. Arsenal](Liverpool_v_Arsenal.png)

I'll explain this even more:
* **26th Minute:** Liverpool goal. We can see a clear increase in Liverpool's sentiment, while Arsenal's suffered an incredibly sharp decline.
* **52nd Minute:** Liverpool goal. This increase is, again, very clear.
* **53rd Minute:** Arsenal goal. This goal came approximately 45 seconds after Liverpool's second, at which point Liverpool's sentiment dropped considerably.
* **56th Minute:** Arsenal goal. An even greater decline in Liverpool sentiment and a sharp increase in Arsenal's.
* **58th Minute:** Arsenal goal. Liverpool sentiment hits rock bottom, at about -23. Weirdly enough, Arsenal's sentiment barely changes.
* **71st minute:** Liverpool goal. Liverpool's sentiment spikes to just above 0, while Arsenal's suffers a minor decline.

Other points of interest:
* **43rd Minute:** Sadio Mane and Mohammed Salah both miss very easy chances for Liverpool, leading to an over-the-top reaction from the Liverpool subreddit.
* **FT:** The end of the game led to a noticeable decrease in sentiment for Liverpool, as they feel that they should have easily won the game. Arsenal remain just above a score of 0.

## Unique Words by Subreddit

I was also curious about what words each subreddit uses, but I wanted to make sure that commonly used soccer phrases, such as cuss words, "banger", "passion", etc. were not included. I took the 1000 most common words per subreddit, not including stopwords, removed any words that showed up in more than one subreddit, and created word clouds for each subreddit using a by-frequency [Word Cloud tool](https://github.com/amueller/word_cloud). AFC Bournemouth's word cloud is shown below:

<h3><center>AFC Bournemouth</center></h3>
![](WordClouds/AFCBournemouthWordCloud.png)

## Textual Analysis of Comments

These graphs are cool and everything, but I also wanted to try to predict something in the model. First, I used the TF-IDF Vectorizer from SciKit and a list of stopwords from NLTK to generate a matrix of weights. Then, I tried to use K-Nearest Neighbors to predict the upvote score of a given comment.

In order for my computer to be able to run the analysis, I only used the first 10,000 comments to test the model. I also split the model to predict upvote scores by subreddit, since each subreddit has a different number of users and therefore a different expected upvote value for the same comment.

```
stopwords = set(stopwords.words('english'))
vec = TfidfVectorizer(stop_words=stopwords, max_features=1000)
vec.fit(comments['body'])
tf_idf_sparse = vec.transform(comments['body'])

X = tf_idf_sparse.todense()
score_actual = comments['score']

model = KNeighborsRegressor(n_neighbors=75)
scaler = StandardScaler()
scaler.fit(X)
X_std = scaler.transform(X)

model.fit(X_std, score_actual)
score_predicted = model.predict(X_std)
```

Using these stopwords, I generated the following 10,000 predictions, with a negative mean squared error of -105.24. The y=x line is shown in red, appearing as almost a flat line.

![K Nearest Neighbors](KNearestNeighbors.png)

As you can see, this model seems to flatten the expected score to a value between 3 and 10, and has trouble predicting the extreme upvoted or downvoted comments. If we perform the same analysis using bigrams instead of individual words, we get the following predictions, with a negative mean squared error of 

![K Nearest Neighbors Bigrams](KNearestNeighborsBigrams.png)

At this point, I tried a few other models, including Random Forest Regressor and Stochastic Gradient Descent, but none of the models yielded better results.

After looking into other, similar projects, I found that most projects take into account the post that a comment is made in, how long after the initial post a comment is made, and the time of day that the post is made, as well as the contents of the comment itself. Even so, these models rarely attempt to predict the score of a comment, but instead predict a "likelihood of success" sort of variable. Due to the various factors included in the score of a reddit comment, it's incredibly difficult to predict how successful a particular comment will do ahead of time.

## Questions?